In [1]:
#Importando librerias
import pandas as pd
import numpy as np
from urllib.request import FancyURLopener
from bs4 import BeautifulSoup
import requests
import urllib

from urllib.request import urlopen
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import time
from random import choice
from urllib.error import HTTPError 
import urllib.parse

Extracción y Preparación de la Data

In [10]:
#Extrayendo la data para análisis
#NOTA: PARA JALAR LA DATA DE OTRA PC, CAMBIAR LA RUTA DE s A LA NUEVA RUTA
s=r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\train.xlsx'
archivo=pd.ExcelFile(s)
hoja=archivo.sheet_names#Extrayendo la data para análisis
#NOTA: PARA JALAR LA DATA DE OTRA PC, CAMBIAR LA RUTA DE s A LA NUEVA RUTA
s=r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\train.xlsx'
archivo=pd.ExcelFile(s)
hoja=archivo.sheet_names
#Extrayendo la data test para análisis y conversion
#NOTA: PARA JALAR LA DATA DE OTRA PC, CAMBIAR LA RUTA DE s A LA NUEVA RUTA
s2=r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\test.xlsx'
archivo2=pd.ExcelFile(s2)
hoja2=archivo2.sheet_names

In [11]:
df=archivo.parse('Hoja de datos')
df.head()

,Category,Topic,Source,Headline,Text,Link
0,Fake,Education,El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",http://www.elruinaversal.com/2017/06/10/rae-in...
1,Fake,Education,Hay noticia,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea...",https://haynoticia.es/la-palabra-haiga-aceptad...
2,Fake,Education,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,http://www.elruinaversal.com/2018/05/06/yordi-...
3,True,Education,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...
4,Fake,Education,Lamula,pretenden aprobar libros escolares con conteni...,Alerta: pretenden aprobar libros escolares con...,https://redaccion.lamula.pe/2018/06/19/memoria...


In [12]:
df2=archivo2.parse('Hoja de datos')
df2.head()

,Category,Topic,Source,Headline,Text,Link
0,Fake,Education,El Ruinaversal,MAESTRA DE *NUMBER* AÑOS QUE TUVO RELACIONES C...,MAESTRA DE *NUMBER* AÑOS QUE TUVO RELACIONES C...,http://www.elruinaversal.com/2017/06/04/maestr...
1,True,Education,Heraldo,Oxford lanza sus propios exámenes de certifica...,Oxford lanza sus propios exámenes de certifica...,https://www.heraldo.es/noticias/sociedad/2017/...
2,True,Education,abc,La RAE estudia incluir «machirulo» en el Dicci...,La RAE estudia incluir «machirulo» en el Dicci...,https://www.abc.es/cultura/abci-estudia-inclui...
3,True,Education,El país,Malala Yousafzai anuncia que estudiará en Oxford,Malala Yousafzai anuncia que estudiará en Oxfo...,https://elpais.com/internacional/2017/08/17/ac...
4,Fake,Education,El Ruinaversal,Nombran a Ricardo Arjona nuevo miembro de la R...,Nombran a Ricardo Arjona nuevo miembro de la R...,http://www.elruinaversal.com/2017/06/25/nombra...


In [13]:
df.shape

(676, 6)

In [14]:
#obtener solo las paginas http de train
df_link=df.drop(['Category','Topic','Source','Headline','Text'] ,axis=1)
#Convirtiendo la data http de tipo objeto a string para uso en URLopener
df_link=df_link['Link'].astype(str)

In [15]:
#Prueba del link de train
textp=df_link.loc[3]
textp

'http://www.eluniversal.com.mx/articulo/nacion/sociedad/2017/02/8/unam-capacitara-maestros-para-aprobar-prueba-pisa'

In [16]:
#obtener solo las paginas http de test
df_link2=df2.drop(['Category','Topic','Source','Headline','Text'] ,axis=1)
#Convirtiendo la data http de tipo objeto a string para uso en URLopener
df_link2=df_link2['Link'].astype(str)

In [17]:
#Prueba del link de test
textp2=df_link2.loc[3]
textp2

'https://elpais.com/internacional/2017/08/17/actualidad/1502965593_437389.html'

In [18]:
#Colocando distintos agentes de búsqueda
user_agents = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    #'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    #'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    #'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9'
]

Funcion de extraccion total no se usa FancyURLopener porque bloqueaba la descarg
Se cambia de urlpone y urllib.request
Se agrega excepción para errores de paginas http inexistentes, se coloca "NA" en esos campos

In [23]:
def agrupa_textos(df_link,rango):
    grupo=[]
    #class MyOpener(FancyURLopener,object):
        #version = choice(user_agents)
    #myopener=MyOpener()
    agent=choice(user_agents)
    
    head = {'User-Agent': agent}

    
    for i in range(0,rango):
                  
        texto= df_link_part1.loc[i]
        try:
            html = urlopen(texto)
        except:
            ptag="NA"
        else:
            #texto_html=myopener.open(urllib.parse.unquote(texto)).read()
            req  =urllib.request.Request(texto, data=None,headers=head)
            response = urllib.request.urlopen(req)
            texto_html=response.read()
            soup=BeautifulSoup(texto_html,"lxml")
            ptag=soup.findAll('p')
        grupo.append(ptag)
           
              
    return(grupo)
    
    

In [9]:
#Inicalizar el grupo, obtener la cantidad de descarga
grupo2=[]
total=len(df_link)-1

Llamada a descarga de archivo de entrenamiento, se realizan descargas por lotes de 10 con intervalos de 1 minuto

In [15]:
parcial=int(total/10)
inicio=0
for i in range(0,parcial):
    cantidad=inicio+10
    df_link_part1 = df_link.iloc[inicio:cantidad]
    df_link_part1 = df_link_part1.reset_index(drop=True)
    
    rango=cantidad-inicio
    
    grupo=agrupa_textos(df_link_part1,rango)
    for s in range(0,rango):
        grupo2.append(grupo[s])
    inicio=inicio+rango
    time.sleep(60)
    
    print(i,"  ",len(grupo2))


0    10
1    20
2    30
3    40
4    50
5    60
6    70
7    80
8    90
9    100
10    110
11    120
12    130
13    140
14    150
15    160
16    170
17    180
18    190
19    200
20    210
21    220
22    230
23    240
24    250
25    260
26    270
27    280
28    290
29    300
30    310
31    320
32    330
33    340
34    350
35    360
36    370
37    380
38    390
39    400
40    410
41    420
42    430
43    440
44    450
45    460
46    470
47    480
48    490
49    500
50    510
51    520
52    530
53    540
54    550
55    560
56    570
57    580
58    590
59    600
60    610
61    620
62    630
63    640
64    650
65    660
66    670


In [28]:
grupo4=[]
total2=len(df_link2)-1

In [37]:
parcial2=int(total2/10)
inicio=0
for i in range(0,parcial2):
    cantidad=inicio+5
    df_link_part1 = df_link2.iloc[inicio:cantidad]
    df_link_part1 = df_link_part1.reset_index(drop=True)
    
    rango=cantidad-inicio
    
    grupo3=agrupa_textos(df_link_part2,rango)
    for s in range(0,rango):
        grupo4.append(grupo3[s])
        inicio=inicio+rango
        time.sleep(60)


8    295


In [38]:
print(len(grupo4))

295


Conversión del Texto a Base de Datos. Formalmente se convierte en Filas con 83 columnas de separacón
Se entregan dos dataframes: El de 83 columnas y otro de 1 sola columna con Join

In [ ]:
df_textos_train=pd.DataFrame(grupo2)
df_texto2=pd.DataFrame(columns=['textos'])
df_texto2['textos'] = df_textos_train.apply(lambda x: ','.join(set(x.astype(str))), axis=1)

In [39]:
df_textos_test=pd.DataFrame(grupo4)
df_texto3=pd.DataFrame(columns=['textos'])
df_texto3['textos'] = df_textos_test.apply(lambda x: ','.join(set(x.astype(str))), axis=1)

In [ ]:
#Probando la data
print(df_texto2.loc[0])


In [40]:
print(df_texto3.loc[0])

textos    <p>Decidí que Alexandria no pisará la cárcel p...
Name: 0, dtype: object


Primer Entregable: Archivos en csv con los textos de las paginas html descargadas. 
Segunda parte: Limpieza:
1.- juntar Columna de Verificación (Noticia Real, Noticia Falsa), eliminar filas con https errados
2.- Retirar textos de formato y de indicación fuera del contexto de la notica
3.- Eliminar artículos y otras palabras irrelevantes para la vectorización. 


In [ ]:
df_textos_train.to_csv(r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\textoseparado_train.csv',encoding='utf-8')
df_texto2.to_csv(r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\textojunto_train.csv',encoding='utf-8')

In [41]:
df_textos_test.to_csv(r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\textoseparado_test.csv',encoding='utf-8')
df_texto3.to_csv(r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\textojunto_test.csv',encoding='utf-8')

SEGUNDA ETAPA: LIMPIEZA DE DATOS 
a- Eliminar paginas fallidas
    a.1- Juntar datos con la variable característica
    a.2- Eliminar filas fallidas

In [21]:
df_http=pd.read_csv(r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\textoseparado_train.csv')
df_objetivo=df.drop(['Link','Topic','Source','Headline','Text'] ,axis=1)    
df_objetivo.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,107,108,109,110,111,112,113,114,115,116
0,0,0,"<p class=""site-title""><a href=""https://www.elr...","<p class=""site-description"">Las noticias en di...",<p>España.- El presidente de la Real Academia ...,<p>Darío señaló que “Lady” servirá para defini...,<p>“Son contadas las palabras del idioma inglé...,<p>La gente podrá decirle Lady a una fémina qu...,<p>Villanueva presentó a los medios la definic...,<p><strong>-Lady</strong></p>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,N,A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,"<p class=""site-title""><a href=""https://www.elr...","<p class=""site-description"">Las noticias en di...",<p>México.- El director de la Secretaría de Ed...,<p>Aurelio señala que decidió contratar a Yord...,<p>“Desafortunadamente el gobierno ya no tiene...,<p>Nuño menciona que el conductor explicará a ...,<p>“Queremos que los niños lleguen a sus casas...,<p>El director menciona que Yordi redactará lo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,N,A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,"<p><img alt=""*"" height=""1"" src=""http://b.score...",<p>Hay un solo presidente</p>,<p>Sigue la campaña negacionista del fujimoris...,"<p>De acuerdo a esta organización, el Minister...",<p>La Cnddhh denunció que esto sería el result...,"<p>En este proceso de revisión, se habrían rea...",<p>La Cnddhh sostuvo que esto tiene la finalid...,<p>Este es el pronunciamiento de la Cnddhh:</p>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df_objetivo.head()

,Category
0,Fake
1,Fake
2,Fake
3,True
4,Fake


In [105]:
df_completo=pd.concat([df_http,df_objetivo],axis=1)
df_completo.shape

(676, 119)

In [121]:
df_prueba = df_completo.dropna(subset=['3'])

In [122]:
df_prueba.shape

(370, 119)

In [125]:
print(len(df_prueba.loc[df_prueba['Category']=='Fake']))

201
